In [ ]:
# import modules and define functions
from pyscf import gto, scf
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt, ticker
import matplotlib as mpl
import os
from scipy import linalg
from functools import reduce
os.environ['OMP_NUM_THREADS'] = "1"
def make_rdm1(mo_coeff, mo_occ):
    mocc = mo_coeff[:,mo_occ>0]
    return np.dot(mocc*mo_occ[mo_occ>0], mocc.conj().T)

# Basis set dependence of charge density

In [ ]:
# build CO with minimal basis
atom = 'C 0 0 0; O 0 0 1.13'
# bss = ['sto-3g', '3-21g', '4-31g', '6-31g', '6-31g*', '6-31g**', '6-31+g**', '6-31++g**']
bss = ['def2-SVP', 'def2-SVPD', 'def2-TZVP', 'def2-TZVPP', 'def2-TZVPPD', 'def2-QZVP', 'def2-QZVPP', 'def2-QZVPPD']
mulliken_charge_dict = {}
loewdin_charge_dict = {}
for basis in bss:
    charge = 0
    spin = 0
    co_0_1 = gto.Mole().build(atom=atom,basis=basis,charge=charge,spin=spin)
    S = co_0_1.intor_symmetric('int1e_ovlp')
    # perform SCF 
    co_0_1_rhf = scf.RHF(co_0_1).run()
    ## calculate PS
    mo_coeff = co_0_1_rhf.mo_coeff
    mo_occ = co_0_1_rhf.mo_occ
    dm = make_rdm1(mo_coeff,mo_occ)
    #######
    #!TODO#
    #######
    PS = 
    #######
    #!TODO#
    #######
    PS_L = 
    ## on which atom are the basis sets
    # number of atomic orbitals
    nao = co_0_1.nao
    # number of basis functions (e.g. 2p orbitals are counted as 1)
    num_bs = co_0_1.nbas
    # find out on which atom is each atomic orbital
    nao_atom_idx = np.zeros(nao)
    count = 0
    for bs_i in range(num_bs):
        for i in range(co_0_1.bas_angular(bs_i)*2+1):
            nao_atom_idx[count] = co_0_1.bas_atom(bs_i)
            count += 1
    # find tr(PS_\mu\mu), mu belongs to A and calculate q_A
    num_atoms = co_0_1.natm
    qs_nuc = [co_0_1.atom_charge(i) for i in range(num_atoms)]
    mulliken_data = {}
    loewdin_data = {}
    for i in range(num_atoms):
        ne_mulliken = np.trace(PS[nao_atom_idx == i].T[nao_atom_idx == i])
        q_mulliken = qs_nuc[i] - ne_mulliken
        ne_loewdin = np.trace(PS_L[nao_atom_idx == i].T[nao_atom_idx == i])
        q_loewdin = qs_nuc[i] - ne_loewdin
        asym = co_0_1.atom_symbol(i)
        mulliken_data[asym] = q_mulliken
        loewdin_data[asym] = q_loewdin
    mulliken_charge_dict[basis] = mulliken_data
    loewdin_charge_dict[basis] = loewdin_data

In [ ]:
# plot
# plot the change of energy 
xlabel = 'Basis sets'
ylabel = 'Charge'
title = 'Basis set effect on Mulliken charge density'
mpl.rcParams['axes.linewidth'] = 3
mpl.rcParams['xtick.major.size'] = 10
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['xtick.major.width'] = 3
mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['ytick.major.width'] = 3
mpl.rcParams['xtick.minor.size'] = 5
mpl.rcParams['xtick.minor.width'] = 3
mpl.rcParams['ytick.minor.size'] = 5
mpl.rcParams['ytick.minor.width'] = 3
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
fig, ax = plt.subplots(figsize=(15,6))
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.1))
ax.set_ylim(-0.5,0.5)
ax.set_xlabel(xlabel, fontsize=16)
ax.set_ylabel(ylabel, fontsize=16)
ax.set_title(title, fontsize=22)
ax.plot(pd.DataFrame(mulliken_charge_dict).T)
ax.legend(['C', 'O'])
plt.show()

In [ ]:
# plot
# plot the change of energy 
xlabel = 'Basis sets'
ylabel = 'Charge'
title = 'Basis set effect on Loewdin charge density'
mpl.rcParams['axes.linewidth'] = 3
mpl.rcParams['xtick.major.size'] = 10
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['xtick.major.width'] = 3
mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['ytick.major.width'] = 3
mpl.rcParams['xtick.minor.size'] = 5
mpl.rcParams['xtick.minor.width'] = 3
mpl.rcParams['ytick.minor.size'] = 5
mpl.rcParams['ytick.minor.width'] = 3
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
fig, ax = plt.subplots(figsize=(15,6))
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.1))
ax.set_ylim(-0.5,0.5)
ax.set_xlabel(xlabel, fontsize=16)
ax.set_ylabel(ylabel, fontsize=16)
ax.set_title(title, fontsize=22)
ax.plot(pd.DataFrame(loewdin_charge_dict).T)
ax.legend(['C', 'O'])
plt.show()